In [218]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras import layers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.optimizers import RMSprop, Adam, Adadelta
from tensorflow.keras.applications import EfficientNetB0, ResNet50
import keras

In [157]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import time as tm
import os
import random

INPUT_SHAPE=[192, 192, 3]
NUM_POINTS=2000
BATCH_SIZE=32
BUFFER_SIZE=32
EPOCHS=50


In [158]:
image_size = 50
image_number = 1000
folder = "../data/SolarPanelSoilingImageDataset/Solar_Panel_Soiling_Image_dataset/PanelImages"
image_width = 192
image_height = 192
ratio = 1

image_width = int(image_width / ratio)
image_height = int(image_height / ratio)

channels = 3

In [159]:
onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

In [160]:
#onlyfiles = onlyfiles[0:1000]

In [161]:
type(onlyfiles)

list

In [162]:
len(onlyfiles)

45754

In [163]:
_file = onlyfiles[0]

In [164]:
x = img_to_array(load_img(folder + "/" + _file))

In [165]:
x.shape

(192, 192, 3)

In [166]:
def gen():
    files = random.choices(onlyfiles, k=1000)
    for _file in files:
        x = img_to_array(load_img(folder + "/" + _file))
        y = np.array(np.float64(_file.split('_')[11]), dtype=np.float64)
        yield x, y

In [167]:
y

array([1])

In [168]:
y = np.random.randint(1, 3, size=1)

In [205]:
def create_model(INPUT_SHAPE):
    """This code uses adapts the most up-to-date version of EfficientNet with NoisyStudent weights to a regression
    problem. Most of this code is adapted from the official keras documentation.

    Returns
    -------
    Model
        The keras model.
    """
    inputs = layers.Input(
        shape=(INPUT_SHAPE)
    )  # input shapes of the images should always be 224x224x3 with EfficientNetB0
    # use the downloaded and converted newest EfficientNet wheights
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(1, name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="EfficientNet")

    return model

In [219]:
def create_model_resnet(INPUT_SHAPE):
    """This code uses adapts the most up-to-date version of EfficientNet with NoisyStudent weights to a regression
    problem. Most of this code is adapted from the official keras documentation.

    Returns
    -------
    Model
        The keras model.
    """
    inputs = layers.Input(
        shape=(INPUT_SHAPE)
    )  # input shapes of the images should always be 224x224x3 with EfficientNetB0
    # use the downloaded and converted newest EfficientNet wheights
    model = ResNet50(include_top=False, input_tensor=inputs, weights="imagenet")
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(1, name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="ResNet50")

    return model

In [220]:

dataset = tf.data.Dataset.from_generator(gen, 
                                         (tf.float32, tf.float32),
                                         output_shapes=(tf.TensorShape(INPUT_SHAPE), tf.TensorShape(None))
                                        ).batch(BATCH_SIZE)


In [221]:
model = create_model_resnet(INPUT_SHAPE)

In [222]:

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, clipvalue=1.0),
    loss='mean_squared_error',
    metrics=['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error']
    )

#iterator = dataset.make_one_shot_iterator()
#x,y = iterator.get_next()
#print("X.shape", x.shape, y.shape)
#print("Predicted", model.predict(x).shape)

print(model.summary())
#model.fit_generator(dataset, epochs=EPOCHS, verbose=2)
model.fit(dataset, epochs=EPOCHS, verbose=1)

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 192, 192, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 198, 198, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 96, 96, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 96, 96, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

32/32 [==============================] - 58s 2s/step - loss: 3.0908 - mean_squared_error: 3.0882 - mean_absolute_error: 1.3116 - mean_absolute_percentage_error: 260665.2545
Epoch 2/50
32/32 [==============================] - 80s 2s/step - loss: 1.7660 - mean_squared_error: 1.7687 - mean_absolute_error: 1.0124 - mean_absolute_percentage_error: 5404.6444
Epoch 3/50
32/32 [==============================] - 79s 2s/step - loss: 1.7812 - mean_squared_error: 1.7805 - mean_absolute_error: 1.0261 - mean_absolute_percentage_error: 8206.7610
Epoch 4/50
32/32 [==============================] - 75s 2s/step - loss: 1.5617 - mean_squared_error: 1.5617 - mean_absolute_error: 0.9559 - mean_absolute_percentage_error: 5428.4410
Epoch 5/50
32/32 [==============================] - 79s 2s/step - loss: 1.5000 - mean_squared_error: 1.4990 - mean_absolute_error: 0.9398 - mean_absolute_percentage_error: 4076.7765
Epoch 6/50
32/32 [==============================] - 75s 2s/step - loss: 1.2715 - mean_squared_error

32/32 [==============================] - 149s 5s/step - loss: 0.6677 - mean_squared_error: 0.6672 - mean_absolute_error: 0.6351 - mean_absolute_percentage_error: 3861.8576
Epoch 46/50
32/32 [==============================] - 142s 4s/step - loss: 0.6511 - mean_squared_error: 0.6516 - mean_absolute_error: 0.6215 - mean_absolute_percentage_error: 1921.7883
Epoch 47/50
32/32 [==============================] - 146s 5s/step - loss: 0.6877 - mean_squared_error: 0.6871 - mean_absolute_error: 0.6180 - mean_absolute_percentage_error: 5296.4003
Epoch 48/50
32/32 [==============================] - 143s 4s/step - loss: 0.5905 - mean_squared_error: 0.5899 - mean_absolute_error: 0.5992 - mean_absolute_percentage_error: 3744.4077
Epoch 49/50
32/32 [==============================] - 158s 5s/step - loss: 0.6557 - mean_squared_error: 0.6555 - mean_absolute_error: 0.6175 - mean_absolute_percentage_error: 47161.8034
Epoch 50/50
32/32 [==============================] - 157s 5s/step - loss: 0.5998 - mean_squ

In [58]:
x = np.random.rand(INPUT_SHAPE[0], INPUT_SHAPE[1] ,INPUT_SHAPE[2])

In [59]:
x.shape

(48, 48, 3)

In [61]:
type(x)

numpy.ndarray

In [62]:
x

array([[[4.66910063e-01, 3.61090573e-01, 9.48553662e-02],
        [6.62111700e-01, 1.74175628e-01, 7.14556595e-01],
        [7.02967659e-01, 2.13492921e-01, 4.74676446e-01],
        ...,
        [7.53478095e-01, 5.74561189e-01, 9.67640665e-01],
        [4.66110866e-01, 4.54458307e-01, 8.47428769e-01],
        [1.57027750e-01, 4.48661479e-02, 8.38039336e-01]],

       [[4.48119296e-01, 6.61709718e-01, 6.60602547e-01],
        [4.64199113e-01, 2.42401277e-01, 1.82837439e-01],
        [3.72568568e-01, 6.82903604e-01, 4.20910256e-01],
        ...,
        [6.46474830e-02, 4.30686829e-01, 3.81097807e-01],
        [8.55893860e-01, 1.60135287e-01, 4.35036260e-01],
        [4.47808753e-01, 2.55096392e-01, 1.02857938e-01]],

       [[1.80354276e-01, 7.57169469e-01, 1.55309552e-01],
        [2.44285969e-01, 8.72724490e-01, 1.43239575e-01],
        [3.42474127e-01, 1.16989677e-01, 2.90516846e-02],
        ...,
        [6.09547281e-01, 3.97402930e-01, 5.21167749e-01],
        [7.15374535e-01, 5.47

In [171]:
import autokeras as ak


In [172]:
# Initialize the image regressor.
reg = ak.ImageRegressor(
    overwrite=True,
    max_trials=10)


In [175]:
# Feed the image regressor with training data.
reg.fit(dataset,
        batch_size=25,
        epochs=10)

ValueError: Cannot take the length of shape with unknown rank.

In [27]:
type(x[0][0])


numpy.float64